### The first cell in this file should always be executed, then one can choose a particular experiment to run.


In [1]:
#Tom Laeven (4247078) en Marnix Huibers (4144015)

from numba import jit
import numpy as np
import matplotlib.pyplot as plt
import functionsArgon3 as fA
#%matplotlib inline

### 1. Standard Simulation (energy checks)
In this experiment the kinetic, potential and total energy are plotted.

In [ ]:
settings = fA.md_settings(TSI=119.8,
                          rho=np.sqrt(2),
                          NumberOfBoxesPerDimension=2,
                          NumberOfTimeSteps=2000,
                          TruncR=3000,
                          h=0.004)
# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed=101)

# The Simulation
for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    r[:,:,q+1] = instance.r
    v[:,:,q+1] = instance.v
    U[q+1] = instance.U
    Ekin[q+1] = instance.Ekin

# Plotting
TimeAxis = np.arange(0,settings.NumberOfTimeSteps+1)
Etot = Ekin + U
plt.title("Energy plot")
plt.xlabel("Time Steps")
plt.ylabel("Energy")
plt.xlim([0, settings.NumberOfTimeSteps])
plt.plot(TimeAxis, Ekin,label="Kinetic Energy")
plt.plot(TimeAxis, U,label="Potential Energy")
plt.plot(TimeAxis, Etot, label="Total Energy")
plt.legend(loc = 5,bbox_to_anchor=(1.51,0.8))
plt.show()

### 2. $C_v(\rho,T)$
In this experiment the heat capacity is calculated. It's important to first qualitatively determine when the system is in equilibrium using the plot of the kinetic energy. The number of time steps needed for equilibrium can be put in the variable Eq.  

In [2]:
settings_list = list()
### Create list of parameters
rho_list = np.array([0.01,0.3,0.8, 1.2])
#rho_list = np.append(rho_list, (0.01, 0.02, 0.03,0.04,0.05,0.06,0.07,0.08,0.09))
T_list = np.array([0.5,1,3])
for T in T_list:
    for rho in rho_list:
        settings_list.append( fA.md_settings(TSI=T*119.8,
                          rho=rho,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=100000,
                          TruncR=3,
                          h=0.004))

In [3]:
def doCvExperiment(settings):
    seed = 101

    Eq = 10000 #Equilibrium time
    Ti = 5000
    NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))

    ####### INITIALISE
    r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)
    ### EQUILIBRATE TEMPERATURE
    labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0]) )
    v[:,:,0] *= labda
    Ekin[0] *= labda*labda
    instance.v = v[:,:,0]

    # The Simulation
    for q in range(0,settings.NumberOfTimeSteps):
        instance = fA.velocity_verlet(instance, settings)
        r[:,:,q+1] = instance.r
        v[:,:,q+1] = instance.v
        U[q+1] = instance.U
        Ekin[q+1] = instance.Ekin
        Virial[q+1] = instance.Virial
        
        ### EQUILIBRATE TEMPERATURE
        if (q < Eq):
            labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
            v[:,:,q+1] *= labda
            Ekin[q+1] *= labda*labda

            instance.v = v[:,:,q+1]

    # CALCULATE RELEVANT QUANTITIES + ERROR
    Cv = np.zeros(NumTi)
    T = np.zeros(NumTi)
    for i in range(0,NumTi):
        EkInt = Ekin[(Eq+Ti*i):(Eq+Ti*(i+1))]
        Cv[i]=(3*settings.N*np.mean(EkInt)*np.mean(EkInt))/(2*np.mean(EkInt)*np.mean(EkInt)-
                                                       3*settings.N*np.var(EkInt))
        T[i] = 2/3 * np.mean(EkInt) / settings.N
        
    TMean = np.mean(T)
    TError = np.std(T) / np.sqrt(NumTi)    
    CvMean = np.mean(Cv)
    CvStd = np.std(Cv)/ np.sqrt(NumTi)
    
    return (settings,TMean,TError,CvMean,CvStd)


In [4]:
import multiprocessing
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=1)
    heatcapacity_data_list = np.asarray(pool.map(doCvExperiment, settings_list)) 
    pool.close()
    pool.join()

##### SAVE DATA #####
filename = "Cv(rho={0}:{1},T={2}:{3})".format(rho_list[0], rho_list[-1] , T_list[0] , T_list[-1])
np.save(filename, heatcapacity_data_list)

In [93]:
##### LOAD DATA #####
cv_data_list = np.load("Cv(rho=0.1:0.09,T=0.7:4.0).npy")
##### MAKE ISOTHERMS #####
isotherms = dict()
for experiment in cv_data_list:
    T = experiment[0].T
    isotherms[T] = list() # might cause due to T-> TSI -> T
for T_Te_Cv_Cve in cv_data_list:
    settings = T_Te_Cv_Cve[0]
    isotherms[settings.T].append([settings.rho, T_Te_Cv_Cve[1:]])

##### PLOT ISOTHERMS #####
TEMPS = sorted(isotherms.keys(),reverse = True)
TEMPS=TEMPS[:2]
for temp in TEMPS:
    line = isotherms[temp]
    Rho = list()
    Cv = list()
    CvE = list()
    for point in line:
        if point[1][3]<20000:
            Rho.append(point[0])
            Cv.append(point[1][2]/864)
            CvE.append(point[1][3]/864)
    Rho = np.array(Rho)
    Cv = np.array(Cv)
    sorted_indices = np.argsort(Rho)
    Rho = Rho[sorted_indices]
    Cv = Cv[sorted_indices]
    print("temp",temp,"Cve",np.max(CvE))
    plt.plot(np.array(Rho), np.array(Cv), label = "T={0}".format(temp))

### PLOT BEAUTIFICATION
plt.xlabel(r"$\rho$",fontsize=20)
plt.ylabel(r"$C_v/N$",fontsize=20)
plt.legend(loc = 2)
#plt.savefig("figures/CvVSrho.pdf".format(Rho[0], Rho[-1] , TEMPS[0] , TEMPS[-1]),bbox_inches = "tight")
plt.show()

temp 4.0 Cve 0.235100486305
temp 1.5 Cve 0.43089445759


### 3. $P\left(\rho\right)$, for different values of T

In [22]:
settings_list = list()
### Create list of parameters
rho_list = np.linspace(0.01, 1.2, 100)
#rho_list = np.append(rho_list, (0.1,0.3,0.5))
T_list = np.array([0.75, 0.99, 1.35,1.5, 2.74])
for T in T_list:
    for rho in rho_list:
        settings_list.append( fA.md_settings(TSI=T*119.8,
                          rho=rho,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=4000,
                          TruncR=3,
                          h=0.004))

In [23]:
def doPressureExperiment(settings):
    seed = 101

    Eq = 2000 #Equilibrium time
    Ti = 500
    NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))

    ####### INITIALISE
    r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)
    ### EQUILIBRATE TEMPERATURE
    labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0]) )
    v[:,:,0] *= labda
    Ekin[0] *= labda*labda
    instance.v = v[:,:,0]

    # The Simulation
    for q in range(0,settings.NumberOfTimeSteps):
        instance = fA.velocity_verlet(instance, settings)
        r[:,:,q+1] = instance.r
        v[:,:,q+1] = instance.v
        U[q+1] = instance.U
        Ekin[q+1] = instance.Ekin
        Virial[q+1] = instance.Virial

        ### EQUILIBRATE TEMPERATURE
        if (q % 20)|(q < Eq):
            labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
            v[:,:,q+1] *= labda
            Ekin[q+1] *= labda*labda

            instance.v = v[:,:,q+1]

    # CALCULATE RELEVANT QUANTITIES + ERROR
    P = np.zeros(NumTi)
    T = np.zeros(NumTi)
    for i in range(0,NumTi):
        VirialInt = Virial[(Eq+Ti*i):(Eq+Ti*(i+1))]
        EkinInt = Ekin[(Eq+Ti*i):(Eq+Ti*(i+1))]

        T[i] = 2/3 * np.mean(EkinInt) / settings.N
        P[i] =  (settings.rho * T[i] - (settings.rho/(3*settings.N)) * np.mean(VirialInt) - 
                2*np.pi*settings.rho* settings.rho/3 *  8/(settings.TruncR**3) ) / (settings.rho* T[i])

    TMean = np.mean(T)
    TError = np.std(T) / np.sqrt(NumTi)
    PMean = np.mean(P)
    PError = np.std(P) / np.sqrt(NumTi)
    return (settings,TMean,TError,PMean,PError)


In [24]:
import multiprocessing
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=2)
    pressure_data_list = np.asarray(pool.map(doPressureExperiment, settings_list)) 
    pool.close()
    pool.join()

##### SAVE DATA #####
filename = "P(rho={0}:{1},T={2}:{3})".format(rho_list[0], rho_list[-1] , T_list[0] , T_list[-1])
np.save(filename, pressure_data_list)

In [5]:
##### LOAD DATA #####
pressure_data_list = np.load("P(rho=0.01:1.2,T=0.75:2.74).npy")
##### MAKE ISOTHERMS #####
isotherms = dict()
for experiment in pressure_data_list:
    T = experiment[0].T
    isotherms[T] = list() # might cause due to T-> TSI -> T
for T_Te_P_Pe in pressure_data_list:
    settings = T_Te_P_Pe[0]
    isotherms[settings.T].append([settings.rho, T_Te_P_Pe[1:]])

##### PLOT ISOTHERMS #####
TEMPS = sorted(isotherms.keys(),reverse = True)
for temp in TEMPS:
    line = isotherms[temp]
    Rho = list()
    P = list()
    for point in line:
        Rho.append(point[0])
        P.append(point[1][2]*point[0]*temp)
    Rho = np.array(Rho)
    P = np.array(P)
    sorted_indices = np.argsort(Rho)
    Rho = Rho[sorted_indices]*(40*1.66e-27)/3.405e-10**3
    P = P[sorted_indices]*119.8*1.38e-23/3.405e-10**3
    plt.plot(np.array(Rho), np.array(P), label = "T={0}".format(temp))

### PLOT BEAUTIFICATION
plt.title(r"Pressure vs. $\rho$ & T ")
plt.xlabel(r"Density $\rho$")
plt.ylabel(r"Pressure")
plt.legend(loc = 5,bbox_to_anchor=(1.31,0.8))
plt.savefig("figures/P(rho={0}:{1},T={2}:{3}).pdf".format(Rho[0], Rho[-1] , TEMPS[0] , TEMPS[-1]),bbox_inches = "tight")
plt.show()